In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests
import xmltodict

C:\Users\GCC\AppData\Local\Temp\ipykernel_25012\3722414010.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
authKey = 'G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D'

## APT 전월세이력 API
def get_apt_rent(sigungu_cd, base_month, authKey = authKey):
    url = f"http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent?serviceKey={authKey}&LAWD_CD={sigungu_cd}&DEAL_YMD={base_month}"
    try:
        response = requests.get(url)
        raw = xmltodict.parse(response.text)
        raw = raw['response']['body']['items']['item']
        return pd.DataFrame(raw)
    except Exception as e:
        return print(f'기준년월: {base_month}, 시군구코드: {sigungu_cd} 결과를 가져오는데 실패했습니다.')

In [3]:
## 법정동코드 불러오기
path = 'C:/Users/GCC/git'
bjd_cd = pd.read_csv(f'{path}/data/DL/법정동코드 전체자료.txt', encoding='cp949', sep='\t', dtype=str)
bjd_cd['시군구코드'] = bjd_cd['법정동코드'].str.slice(0, 5)
bjd_cd['시도코드'] = bjd_cd['법정동코드'].str.slice(0, 2)

In [4]:
## 조회조건 정의
sigungu_cd_ls = (bjd_cd[((bjd_cd['시도코드'] + '000') != bjd_cd['시군구코드'])&(bjd_cd['폐지여부'] == '존재')&(bjd_cd['시도코드'] == '11')]['시군구코드']).unique()
base_month_ls = pd.date_range('2011-01-01', '2024-01-31', freq='M').strftime('%Y%m').tolist()

In [5]:

import concurrent.futures
import threading
_ls = []
lock = threading.Lock()
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    for sigungu_cd in sigungu_cd_ls:
        print(f'{sigungu_cd} 시작')
        future_month = {executor.submit(get_apt_rent, sigungu_cd, month): month for month in base_month_ls}
        for future in concurrent.futures.as_completed(future_month):
            dataset = future.result()
            lock.acquire()
            _ls.append(dataset)
            lock.release()
        print(f"done {sigungu_cd}")
rent_df = pd.concat(_ls)

11110 시작
done 11110
11140 시작
done 11140
11170 시작
done 11170
11200 시작
done 11200
11215 시작
done 11215
11230 시작
done 11230
11260 시작
done 11260
11290 시작
done 11290
11305 시작
기준년월: 201302, 시군구코드: 11305 결과를 가져오는데 실패했습니다.
기준년월: 201212, 시군구코드: 11305 결과를 가져오는데 실패했습니다.
기준년월: 201303, 시군구코드: 11305 결과를 가져오는데 실패했습니다.
기준년월: 201301, 시군구코드: 11305 결과를 가져오는데 실패했습니다.
done 11305
11320 시작
done 11320
11350 시작
done 11350
11380 시작
done 11380
11410 시작
done 11410
11440 시작
done 11440
11470 시작
done 11470
11500 시작
done 11500
11530 시작
done 11530
11545 시작
done 11545
11560 시작
done 11560
11590 시작
done 11590
11620 시작
done 11620
11650 시작
done 11650
11680 시작
done 11680
11710 시작
done 11710
11740 시작
done 11740


In [6]:
path = 'C:/Users/GCC/git'
rent_df.to_csv(f'{path}/data/DL/전월세내역_서울.csv', index=False, encoding='utf-8-sig')